<a href="https://colab.research.google.com/github/YusakuH02/myRepository/blob/main/langchain_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install openai langchain llama-index google-search-results faiss-cpu

In [ ]:
!pip install langchain openai
!pip install faiss-cpu tiktoken chromadb

In [ ]:
!pip install langchain langchain-community

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key="***your key***"
)

response = llm([HumanMessage(content="LangChainは動作していますか")])
print(response.content)


In [ ]:
import requests

# URL定義（問題と解答）
url_202406_Q = "https://www.ipa.go.jp/shiken/mondai-kaiotu/m42obm000000afqx-att/2024r06a_ap_pm_qs.pdf"
url_202406_A = "https://www.ipa.go.jp/shiken/mondai-kaiotu/m42obm000000afqx-att/2024r06a_ap_pm_ans.pdf"

# 問題PDFのダウンロード
response_202406_Q = requests.get(url_202406_Q)
with open("202406_ap_pm_question.pdf", "wb") as f:
    f.write(response_202406_Q.content)

# 解答PDFのダウンロード
response_202406_A = requests.get(url_202406_A)
with open("202406_ap_pm_answer.pdf", "wb") as f:
    f.write(response_202406_A.content)

fitzでテキスト分割

In [ ]:
!pip install PyMuPDF

import fitz  # PyMuPDF
x = 0
def extract_text_from_pdf(x, pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for i, page in enumerate(doc):
        if i < x:
            continue  # 不要なページのスキップ
        text += page.get_text()
    return text

#pdf_text_202406_Q = extract_text_from_pdf(6, "202406_ap_pm_question.pdf")
pdf_text_202406_A = extract_text_from_pdf(0, "202406_ap_pm_answer.pdf")

OCRで画像認識して問題pdfをテキストに

In [ ]:
!apt install poppler-utils
!pip install pdf2image pytesseract

In [ ]:
!apt install tesseract-ocr-jpn

In [ ]:
import pytesseract
from pdf2image import convert_from_path
import os

# OCR対象のPDFファイル
pdf_path = "202406_ap_pm_question.pdf"

# 画像に変換
images = convert_from_path(pdf_path)

# OCRでテキスト抽出
ocr_text = ""
for i, image in enumerate(images):
    text = pytesseract.image_to_string(image, lang="jpn")  # 日本語対応
    ocr_text += f"\n--- Page {i + 1} ---\n{text}"


In [ ]:
pdf_text_202406_Q = ocr_text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
)

docs_202406_Q = text_splitter.create_documents([pdf_text_202406_Q])
docs_202406_A = text_splitter.create_documents([pdf_text_202406_A])

In [ ]:
print("1つ目のチャンク内容:")
#print(docs_202406_Q[0].page_content)
print(docs_202406_A[0].page_content)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key="***your key***"
)

In [ ]:
vectorstore_q = FAISS.from_documents(docs_202406_Q, embeddings)
vectorstore_a = FAISS.from_documents(docs_202406_A, embeddings)

In [ ]:
# 一定文字数以下 or 空白が多すぎるチャンクを除外
filtered_docs = [
    d for d in docs_202406_Q
    if len(d.page_content.strip()) > 200 and "問" in d.page_content
]

In [ ]:
for i, d in enumerate(docs_202406_Q[:10]):
    print(f"\n--- Chunk {i} ---\n")
    print(d.page_content[:800])

In [ ]:
query = "プログラミング分野の問題を一題出題してください"
retrieved_question = vectorstore_q.similarity_search(query, k=1)[0].page_content

print("📝 問題：\n", retrieved_question)

In [ ]:
user_answer = input("あなたの回答を入力してください：\n")

In [ ]:
retrieved_answer = vectorstore_a.similarity_search(retrieved_question, k=1)[0].page_content

feedback_prompt = f"""
以下は応用情報技術者試験の問題とその模範解答です。

【問題】
{retrieved_question}

【模範解答】
{retrieved_answer}

【ユーザーの回答】
{user_answer}

このユーザーの回答が適切かどうかを評価してください。不足があれば解説と修正案を提示してください。また、関連する知識やポイントがあれば追加で教えてください。
"""

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key="***your key***"
)
response = llm([HumanMessage(content=feedback_prompt)])

print("フィードバック：\n", response.content)


↓問題の概要について答えるだけ

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

qa_chain = RetrievalQA.from_chain_type(
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key="***your key***"
),
    retriever=vectorstore.as_retriever(),
)

result = qa_chain.run("問3の内容を簡単に説明してください")

print("回答：")
print(result)
